In [1]:
from Dissects.io import (load_NDskl, 
                         load_image,
                         load_skeleton,
                         save_skeleton,
                         save_fits
                        )
from Dissects.image import (z_project,
                            thinning,
                            dilation)
from Dissects.geometry import Skeleton


from Dissects.analysis.analysis import (general_analysis,
                              cellstats
                              )
from skimage import morphology
from Dissects.analysis.analysis_3D_apical import (update_geom,
                                         junction_intensity,
                                         morphology_analysis)

In [2]:
import os

from skimage import io
import numpy as np
import pandas as pd
import copy
import sys
import matplotlib.pyplot as plt
sys.setrecursionlimit(10000)

import plotly.express as px

from scipy.ndimage.morphology import (binary_fill_holes,
                                      binary_dilation,
                                     )
from scipy import ndimage
import scipy as sci
# %matplotlib notebook
%load_ext snakeviz

In [16]:
directory="/media/admin-suz/Datas/Comparative-analysis_Manu-Tatiana/Squelettes_Disperse"
filename="T4-T5_20210429_sqhRFPt3B-EcadGFP_leg_FastAiSc-Opt1160x1160_pixelsize0v09_Speed7_GB1.fits_c2.85e+03.up.NDskl.BRK.NDskl.a.NDskl"

In [17]:
# get original image
img0 = io.imread(os.path.join("/media/admin-suz/Datas/Comparative-analysis_Manu-Tatiana/Squelettes_Disperse", 
                              "T3-T4_20210429_sqhRFPt3B-EcadGFP_leg_FastAiSc-Opt1160x1160_pixelsize0v09_Speed7_GB1.fits_c2.1e+03.up.NDskl-CLEAN.tif"))

In [18]:
cp, fil, point, specs = load_NDskl(os.path.join(directory, filename))

# #create skeleton object
skel = Skeleton(cp, fil, point, specs)

/home/admin-suz/miniconda3/lib/python3.8/site-packages/Dissects-0.0.1-py3.8.egg/Dissects/geometry/skeleton.py:24: UserWarning:

Has pixel can be half pixel from DisPerSE output,                   we take the integer part of xyz values. 



In [19]:
#Clean skeleton
skel.remove_lonely_cp()
skel.remove_free_filament()

/home/admin-suz/miniconda3/lib/python3.8/site-packages/Dissects-0.0.1-py3.8.egg/Dissects/geometry/skeleton.py:60: UserWarning:

specs dictionnary is not updated, ncrit value is not updated.



In [20]:
skel.specs

{'ndims': 3,
 'bbox': array([0., 0., 0.]),
 'bbox_delta': array([ 212., 1052.,  106.]),
 'ncrit': 3746,
 'nfil': 3582}

In [21]:
skeleton_image = skel.create_binary_image()
img_binary_3d = morphology.skeletonize(skeleton_image)

# 3D vertex detection

In [22]:
from Dissects.segmentation.seg_3D_apical import (generate_segmentation)

In [23]:
face_df, edge_df, vert_df = generate_segmentation(skel, clean=False)


/home/admin-suz/miniconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



find cell


In [24]:
vert_df_save = vert_df.copy(deep=True)
edge_df_save = edge_df.copy(deep=True)
face_df_save = face_df.copy(deep=True)

In [26]:
vert_df.to_csv('/media/admin-suz/Datas/Comparative-analysis_Manu-Tatiana/Squelettes_Disperse/T4-T5/vert_df.csv')
edge_df.to_csv('/media/admin-suz/Datas/Comparative-analysis_Manu-Tatiana/Squelettes_Disperse/T4-T5/edge_df.csv')
face_df.to_csv('/media/admin-suz/Datas/Comparative-analysis_Manu-Tatiana/Squelettes_Disperse/T4-T5/face_df.csv')

In [62]:
vert_df = pd.read_csv('/media/admin-suz/Datas/Comparative-analysis_Manu-Tatiana/Squelettes_Disperse/T2-T3/vert_df.csv', index_col='Unnamed: 0')
edge_df = pd.read_csv('/media/admin-suz/Datas/Comparative-analysis_Manu-Tatiana/Squelettes_Disperse/T2-T3/edge_df.csv', index_col='Unnamed: 0')
face_df = pd.read_csv('/media/admin-suz/Datas/Comparative-analysis_Manu-Tatiana/Squelettes_Disperse/T2-T3/face_df.csv', index_col='Unnamed: 0')

# get original image
img0 = io.imread(os.path.join("/media/admin-suz/Datas/Comparative-analysis_Manu-Tatiana/Squelettes_Disperse", 
                              "T2-T3_20210429_sqhRFPt3B-EcadGFP_leg_FastAiSc-Opt1160x1160_pixelsize0v09_Speed7_GB2.fits_c1.35e+03.up.NDskl.a.NDskl-CLEAN.tif"))

In [60]:
vertex_image = np.zeros(img0.shape)
s = ndimage.generate_binary_structure(3,3)

vertex_image[vert_df.z.to_numpy(),
                     vert_df.y.to_numpy(),
                     vert_df.x.to_numpy()]=1
vertex_image = ndimage.morphology.binary_dilation(vertex_image, structure=s)
vertex_image = ndimage.morphology.binary_dilation(vertex_image, structure=s)

vertex_image = ~vertex_image
from tifffile import tifffile
tifffile.imsave('/media/admin-suz/Datas/Comparative-analysis_Manu-Tatiana/Squelettes_Disperse/T4-T5/vertex.tif',
                vertex_image)

In [63]:
import plotly.graph_objects as go

z0, y0,x0 = np.where(img0>0)
fond = go.Scatter3d(x=x0,
                      y=y0,
                      z=z0,
                      mode='markers',
                     marker=dict(
                        size=2,
                        color='black',
                        opacity=0.1
                     )
    )


vertex_isolated = go.Scatter3d(x=vert_df.x,
                      y=vert_df.y,
                      z=vert_df.z,
                      mode='markers',
                     marker=dict(
                        size=2,
                        color='red', 
                        opacity=0.5
                     )
    )



fig = go.Figure(data=[fond, vertex_isolated], )
fig.update_layout(title='Filament', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )


from matplotlib.colors import ListedColormap


zs,ys,xs = vert_df.loc[edge_df['srce']][list('zyx')].values.flatten(order='F').reshape(3,edge_df.shape[0])
zt,yt,xt = vert_df.loc[edge_df['trgt']][list('zyx')].values.flatten(order='F').reshape(3,edge_df.shape[0])

rand = (np.random.rand(len(zs),3)*255).astype('int')
rand[0] = 0
cmap_rand = ListedColormap(rand)
for i in range(len(zs)):
    
    fig.add_trace(
            go.Scatter3d(
                x=[xs[i], xt[i]],
                y=[ys[i], yt[i]],
                z=[zs[i], zt[i]],
                mode='lines',
                line={"color":rand[[i,i,i]],
                      "width":10,
                     },                
            )
        )


fig['layout'].update(scene=dict(aspectmode="data"), showlegend=False)
fig.show()


In [64]:
import plotly.graph_objects as go
from matplotlib.colors import ListedColormap
rand = (np.random.rand(200,3)*255).astype('int')
rand[0] = 0
cmap_rand = ListedColormap(rand)


plot_face=[]
import matplotlib as mpl
import random 
fill_colors=list(mpl.colors.cnames.values())
random.shuffle(fill_colors)
for f in (np.unique(edge_df.face)[1:]):
    
    try:
        edges= edge_df[edge_df.face==f]
        nb_vert = len(edges)
        vert_order = [edges.iloc[0].srce]
        for i in range(nb_vert):
            vert_order.append(edges[edges.trgt==vert_order[-1]]['srce'].to_numpy()[0])
        zs=vert_df.loc[vert_order].z.to_numpy()
        ys=vert_df.loc[vert_order].y.to_numpy()
        xs=vert_df.loc[vert_order].x.to_numpy()
        plot_face.append(dict(
            type='scatter3d',
            mode='lines',
            x=xs,
            y=ys,
            z=zs,
            name='',
            surfaceaxis=2, # add a surface axis ('1' refers to axes[1] i.e. the y-axis)
            surfacecolor=fill_colors[f],
            line=dict(
                color='black',
                width=4
            ),
        ))
    except:
        print(f)
    
    
    
plot_face.append(fond)
plot_face.append(vertex_isolated)
fig = go.Figure(data=plot_face, )
fig.update_layout(title='Filament', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )

fig['layout'].update(scene=dict(aspectmode="data"), showlegend=False)
fig.show()

In [15]:
efsefsefsf

NameError: name 'efsefsefsf' is not defined

In [ ]:
vert_df = vert_df_save.copy(deep=True)
edge_df = edge_df_save.copy(deep=True)
face_df = face_df_save.copy(deep=True)

# Analyse

In [39]:
from Dissects.analysis.analysis_3D_apical import (update_geom,
                                         junction_intensity,
                                         morphology_analysis)

In [40]:
# morphometry analysis
update_geom(face_df, edge_df, vert_df)
morphology_analysis(face_df, edge_df, vert_df)

In [ ]:
# import image original
# get original image

img_myosin = io.imread(os.path.join(directory, 
                              "C1-20171214_sqh-GFP_ap-alpha-cat-RFP_WP2h-001-dorsal_AiSc_green_bin4.tif"))


## Mesure d'intensité au niveau des jonctions

In [ ]:
junction_intensity(img_myosin, edge_df, 'myosin_intensity')

In [41]:
# plot prot intensity

import plotly.graph_objects as go

z0, y0,x0 = np.where(img_binary_3d==1)
fond = go.Scatter3d(x=x0,
                      y=y0,
                      z=z0,
                      mode='markers',
                     marker=dict(
                        size=2,
                        color='black',
                        opacity=0.1
                     )
    )


vertex_isolated = go.Scatter3d(x=vert_df.x,
                      y=vert_df.y,
                      z=vert_df.z,
                      mode='markers',
                     marker=dict(
                        size=2,
                        color='red', 
                        opacity=1
                     )
    )



fig = go.Figure(data=[vertex_isolated], )
fig.update_layout(title='Filament', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )



zs,ys,xs = vert_df.loc[edge_df['srce']][list('zyx')].values.flatten(order='F').reshape(3, edge_df.shape[0])
zt,yt,xt = vert_df.loc[edge_df['trgt']][list('zyx')].values.flatten(order='F').reshape(3, edge_df.shape[0])
c = (edge_df.myosin_intensity/np.max(edge_df.myosin_intensity)*255).astype('int')

import pylab as pl
cmap = pl.cm.coolwarm


for i in range(len(zs)):
    
    fig.add_trace(
            go.Scatter3d(
                x=[xs[i], xt[i]],
                y=[ys[i], yt[i]],
                z=[zs[i], zt[i]],
                mode='lines',
                line={"color":[cmap(c[i]), cmap(c[i])],
                      "width":10,
                     },                
            )
        )


fig['layout'].update(scene=dict(aspectmode="data"), showlegend=False)
fig.show()

AttributeError: 'DataFrame' object has no attribute 'myosin_intensity'

In [42]:
# plot area

import plotly.graph_objects as go

import pylab as pl
cmap = pl.cm.cividis
# face_df['area']-=face_df.area.min()
# area=face_df.area.values/np.max(face_df.area.values)*255
plot_face=[]
for f in (np.unique(edge_df.face)[1:]):
#     print('rgb'+str(cmap(int(ar[f]))))
    edges= edge_df[edge_df.face==f]
    nb_vert = len(edges)
    vert_order = [edges.iloc[0].srce]
    for i in range(nb_vert):
        vert_order.append(edges[edges.trgt==vert_order[-1]]['srce'].to_numpy()[0])
    zs=vert_df.loc[vert_order].z.to_numpy()
    ys=vert_df.loc[vert_order].y.to_numpy()
    xs=vert_df.loc[vert_order].x.to_numpy()
    plot_face.append(dict(
        type='scatter3d',
        mode='lines',
        x=xs,
        y=ys,
        z=zs,
        name='',
        surfaceaxis=2, # add a surface axis ('1' refers to axes[1] i.e. the y-axis)
        surfacecolor='rgb'+str(cmap(int(face_df['area'].iloc[f-1]))[0:3]),
#             surfacecolor=fill_colors[int(ar[f]/2)],
        line=dict(
            color='black',
            width=4
        ),
    ))
   
    
fig = go.Figure(data=plot_face, )
fig.update_layout(title='Filament', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )

fig['layout'].update(scene=dict(aspectmode="data"), showlegend=False)
fig.show()

In [ ]:
fsgdfgdg

# Analyse au niveau de la face

In [43]:
from Dissects.analysis.analysis_3D_apical import face_intensity, compute_normal

In [44]:
# define pixel size [x,y,z] in µm
pixel_size = {'x':0.1832311, 'y':0.1832311, 'z':0.2201818}
# Largeur d'épaississement de la surface de la cellule
# Cette valeur est ajoutée au dessus et en dessous du plan de la face
thicken = 0.4
# compute_normal(face_df, edge_df, vert_df)
face_intensity(img_myosin, face_df, edge_df, vert_df, thicken, pixel_size)


NameError: name 'img_myosin' is not defined

In [ ]:
# plot area

import plotly.graph_objects as go

import pylab as pl
cmap = pl.cm.cividis
face_df['intensity']-=face_df.intensity.min()
face_df['intensity_norm']=face_df.intensity.values/np.max(face_df.intensity.values)*255
plot_face=[]
for f in (np.unique(edge_df.face)[1:]):
#     print('rgb'+str(cmap(int(ar[f]))))
    edges= edge_df[edge_df.face==f]
    nb_vert = len(edges)
    vert_order = [edges.iloc[0].srce]
    for i in range(nb_vert):
        vert_order.append(edges[edges.trgt==vert_order[-1]]['srce'].to_numpy()[0])
    zs=vert_df.loc[vert_order].z.to_numpy()
    ys=vert_df.loc[vert_order].y.to_numpy()
    xs=vert_df.loc[vert_order].x.to_numpy()
    plot_face.append(dict(
        type='scatter3d',
        mode='lines',
        x=xs,
        y=ys,
        z=zs,
        name='',
        surfaceaxis=2, # add a surface axis ('1' refers to axes[1] i.e. the y-axis)
        surfacecolor='rgb'+str(cmap(int(face_df['intensity_norm'].iloc[f-1]))[0:3]),
#             surfacecolor=fill_colors[int(ar[f]/2)],
        line=dict(
            color='black',
            width=4
        ),
    ))
   
    
fig = go.Figure(data=plot_face, )
fig.update_layout(title='Filament', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )

fig['layout'].update(scene=dict(aspectmode="data"), showlegend=False)
fig.show()

In [ ]:
import plotly.graph_objects as go
from matplotlib.colors import ListedColormap
rand = (np.random.rand(200,3)*255).astype('int')
rand[0] = 0
cmap_rand = ListedColormap(rand)



z0, y0,x0 = np.where(img_binary_3d==1)
fond = go.Scatter3d(x=x0,
                      y=y0,
                      z=z0,
                      mode='markers',
                     marker=dict(
                        size=2,
                        color='black',
                        opacity=1
                     )
    )

z0, y0,x0 = np.where(img_plane==1)
face = go.Scatter3d(x=x0,
                      y=y0,
                      z=z0,
                      mode='markers',
                     marker=dict(
                        size=2,
                        color='blue', 
                        opacity=0.2
                     )
    )


plot_face=[]
plot_face.append(fond)
plot_face.append(face)
fig = go.Figure(data=plot_face, )
fig.update_layout(title='Filament', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )

fig['layout'].update(scene=dict(aspectmode="data"), showlegend=False)
fig.show()

In [ ]:
from tyssue import Sheet
from tyssue import SheetGeometry
face_df = pd.DataFrame(index=np.arange(1, np.max(edge_df.face)))
# remove_edge = edge_df[edge_df.face==-1].index
# edge_df.drop(remove_edge, axis=0, inplace=True)
# edge_df.reset_index(inplace=True, drop=True)
data={'vert':vert_df, 'edge':edge_df, 'face':face_df}
sheet = Sheet('spherical', data)
sheet.reset_index()
sheet.reset_topo()
SheetGeometry.update_all(sheet)

from tyssue.draw import sheet_view
fig, ax = sheet_view(sheet,list('yz'), mode='2D')

In [ ]:
from tyssue.draw import sheet_view
fig, ax = sheet_view(sheet, mode='3D')
fig